<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [ ]:
from ultralytics import YOLO
import torch
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import yaml
import psutil
from pathlib import Path
import json
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PP

mi deve salvare i pth nella cartella /content/drive/MyDrive/pth_person_detect

Ti aiuto a modificare il percorso di salvataggio dei modelli.
Claude can make mistakes. Please double-check responses.


Smoker Detection Training with YOLOv8
from ultralytics import YOLO
import torch
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import yaml
import psutil
from pathlib import Path
import json
from datetime import datetime

class ModelCallback:
    def __init__(self, save_dir):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.best_map = 0
        self.training_history = []
        
    def on_train_epoch_end(self, trainer):
        # Get current metrics
        metrics = trainer.metrics
        epoch = trainer.epoch
        
        # Extract mAP50-95
        current_map = metrics.get('metrics/mAP50-95(B)', 0)
        
        # Save metrics history
        self.training_history.append({
            'epoch': epoch,
            'mAP': current_map,
            'metrics': metrics
        })
        
        # Save history to JSON
        with open(self.save_dir / 'training_history.json', 'w') as f:
            json.dump(self.training_history, f, indent=4)
        
        # Save model if it's the best so far
        if current_map > self.best_map:
            self.best_map = current_map
            save_path = self.save_dir / f'best_model_map_{current_map:.4f}.pth'
            torch.save(trainer.model.state_dict(), save_path)
            
            # Save a symlink to the best model
            best_link = self.save_dir / 'best_model.pth'
            if best_link.exists():
                best_link.unlink()
            best_link.symlink_to(f'best_model_map_{current_map:.4f}.pth')
            
        # Always save last model
        last_model_path = self.save_dir / f'last_model_epoch_{epoch}.pth'
        torch.save(trainer.model.state_dict(), last_model_path)
        
        # Update symlink to last model
        last_link = self.save_dir / 'last_model.pth'
        if last_link.exists():
            last_link.unlink()
        last_link.symlink_to(f'last_model_epoch_{epoch}.pth')

In [ ]:
def check_system_usage():
    """Monitor system resources"""
    print("\nSystem Usage:")
    print(f"CPU Usage: {psutil.cpu_percent()}%")
    print(f"RAM Usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

In [ ]:
def create_dataset_yaml(train_path, val_path, num_classes=2):
    """Create YAML configuration file for YOLOv8"""
    data = {
        'train': train_path,
        'val': val_path,
        'names': {
            0: 'smoker',
            1: 'non_smoker'
        },
        'nc': num_classes
    }
    
    yaml_path = Path('dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)
    
    return str(yaml_path)

In [ ]:
def convert_coco_to_yolo(coco_annotation_file, image_dir, output_dir):
    """Convert COCO format annotations to YOLO format with progress bar"""
    from pycocotools.coco import COCO
    
    print(f"\nLoading COCO annotations from {coco_annotation_file}")
    coco = COCO(coco_annotation_file)
    
    # Create output directories
    output_dir = Path(output_dir)
    (output_dir / 'labels').mkdir(parents=True, exist_ok=True)
    (output_dir / 'images').mkdir(parents=True, exist_ok=True)
    
    # Get total number of images for progress bar
    img_ids = coco.getImgIds()
    total_images = len(img_ids)
    
    print(f"Converting {total_images} images and annotations...")
    
    # Process each image with tqdm progress bar
    for img_id in tqdm(img_ids, desc="Converting annotations", unit="image"):
        img_info = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        
        # Prepare paths
        label_file = Path(img_info['file_name']).stem + '.txt'
        label_path = output_dir / 'labels' / label_file
        img_src = Path(image_dir) / img_info['file_name']
        img_dst = output_dir / 'images' / img_info['file_name']
        
        # Copy image
        if img_src.exists():
            from shutil import copy2
            copy2(str(img_src), str(img_dst))
        
        # Convert annotations
        with open(label_path, 'w') as f:
            for ann in anns:
                # Convert category_id: 0 for smoker, 1 for non_smoker
                category_id = 0 if ann['category_id'] == 1 else 1
                
                # Convert bbox to YOLO format
                x, y, w, h = ann['bbox']
                x_center = (x + w/2) / img_info['width']
                y_center = (y + h/2) / img_info['height']
                w = w / img_info['width']
                h = h / img_info['height']
                
                # Write YOLO format line
                f.write(f"{category_id} {x_center} {y_center} {w} {h}\n")

In [ ]:
def train_model(yaml_path, save_dir, epochs=100, imgsz=640, batch=16):
    """Train YOLOv8 model with progress monitoring and model saving"""
    print("\nInitializing YOLOv8 model...")
    model = YOLO('yolov8n.pt')
    
    # Initialize callback with the specified save directory
    callback = ModelCallback(save_dir)
    
    print("\nStarting training...")
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=0 if torch.cuda.is_available() else 'cpu',
        patience=50,  # Early stopping patience
        save=True,
        project=str(save_dir),
        name='train',
        exist_ok=True,
        callbacks=[callback]
    )
    
    print(f"\nTraining completed. Models saved in {save_dir}")
    print(f"Best mAP: {callback.best_map:.4f}")
    
    return model, callback.best_map

In [ ]:
if __name__ == "__main__":
    print("Starting Smoker Detection Training Pipeline")
    
    # Paths
    train_image_dir = '/content/drive/MyDrive/Photo/train'
    train_coco_annotation_file = '/content/drive/MyDrive/Photo/train/_annotations.coco.json'
    valid_image_dir = '/content/drive/MyDrive/Photo/valid'
    valid_coco_annotation_file = '/content/drive/MyDrive/Photo/valid/_annotations.coco.json'
    
    # Set save directory
    save_dir = Path('/content/drive/MyDrive/pth_person_detect')
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Add timestamp to create a unique subdirectory for this training run
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = save_dir / f'run_{timestamp}'
    
    # Convert datasets
    print("\nPreparing datasets...")
    convert_coco_to_yolo(train_coco_annotation_file, train_image_dir, 'dataset/train')
    convert_coco_to_yolo(valid_coco_annotation_file, valid_image_dir, 'dataset/valid')
    
    # Create YAML configuration
    yaml_path = create_dataset_yaml('dataset/train', 'dataset/valid')
    
    # Train model
    model, best_map = train_model(yaml_path, run_dir, epochs=100)
    
    # Final system usage check
    check_system_usage()
    
    print(f"\nTraining pipeline completed successfully!")
    print(f"Best model achieved mAP: {best_map:.4f}")
    print(f"Models saved in: {run_dir}")